<img src="images/non_max.png" style="height: 500px">

In [1]:
import numpy as np
import cv2

- overlapThresh => kutuların kesişimlerini bulduktan sonra bir thresholddan geçirilir.

In [2]:
def non_max_suppression(boxes, probs = None, overlapThresh = 0.3):
    
    if len(boxes) == 0: # for precautionary purposes
        return []
    
    if boxes.dtype.kind == "i": # for precautionary purposes
        boxes = boxes.astype("float") # if type int => convert float
        
    # initial box values
    x1 = boxes[:,0]
    y1 = boxes[:,1]

    # finite box values
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    
    # find area
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    
    idxs = y2 # right bottom
    
    # probability values
    if probs is not None:
        idxs = probs
        
    # sorting index
    idxs = np.argsort(idxs) # sorting index by probability values 
    
    pick = [] # chosen boxes
    
    while len(idxs) > 0:
        
        last = len(idxs) - 1 # last index
        i = idxs[last] # last value
        pick.append(i) # add last value to box named pick.
        
        # max and min (x,y)
        x1_max = np.maximum(x1[i], x1[idxs[:last]])
        y1_max = np.maximum(y1[i], y1[idxs[:last]])
        x2_min = np.minimum(x2[i], x2[idxs[:last]])
        y2_min = np.minimum(y2[i], y2[idxs[:last]])
        
        # find weight and height of the rectangle.
        w = np.maximum(0, x2_min - x1_max + 1)
        h = np.maximum(0, y2_min - y1_max + 1)
        
        # overlap = intersection / union
        overlap = (w*h)/area[idxs[0:last]]
        
        # delete values below a certain threshold value
        # np.where => which index? :)
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))
        
    return boxes[pick].astype("int")